# Citibike Project - Normalizing the Datasets

Normalizing the bike share datsets from `03_normalizing_datasets.ipynb` to prepare for SQL database creation.

## Imports

In [77]:
import pandas as pd
import numpy as np

import polars as pl

import pyarrow as pa
import pyarrow.parquet as pq

import feather

from matplotlib import pyplot as plt

import os

## Files that we'll be working on: 
Two .CSVs, `group1` is ~10Gb and `group2` is ~20Gb

In [2]:
group1_location='/Users/sra/files/projects/citibike_project/combined/group1_combined/group1.csv'
group2_location='/Users/sra/files/projects/citibike_project/combined/group2_combined/group2.csv'

# Let's explore `group1` first:

In [39]:
q=(
    pl.scan_csv(group1_location,ignore_errors=True,try_parse_dates=True)
)

group1_pl=q.collect()

In [40]:
group1_pl.head()

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
str,str,datetime[μs],datetime[μs],str,f64,str,f64,f64,f64,f64,f64,str
"""26A3DC47FE0EA3…","""docked_bike""",2021-05-13 12:48:08,2021-05-13 13:07:37,"""Broadway & W 2…",6173.08,"""E 2 St & Avenu…",5515.02,40.742868,-73.989186,40.722174,-73.983688,"""member"""
"""A99F2E1D627B08…","""docked_bike""",2021-05-16 08:30:13,2021-05-16 08:45:47,"""46 Ave & 5 St""",6286.02,"""34th Ave & Ver…",6873.01,40.74731,-73.95451,40.765354,-73.939863,"""member"""
"""43E79A45997B73…","""docked_bike""",2021-05-01 08:38:14,2021-05-01 08:54:27,"""46 Ave & 5 St""",6286.02,"""34th Ave & Ver…",6873.01,40.74731,-73.95451,40.765354,-73.939863,"""member"""
"""8B3CC649F4F588…","""docked_bike""",2021-05-09 08:12:31,2021-05-09 08:27:05,"""46 Ave & 5 St""",6286.02,"""34th Ave & Ver…",6873.01,40.74731,-73.95451,40.765354,-73.939863,"""member"""
"""7101C98F057486…","""docked_bike""",2021-05-27 07:52:27,2021-05-27 08:09:01,"""E 123 St & Lex…",7636.05,"""1 Ave & E 78 S…",7020.09,40.802926,-73.9379,40.771404,-73.953517,"""member"""


In [41]:
group1_pl.groupby('start_station_name').count()\
.sort('count',descending=True)\
.head()

start_station_name,count
str,u32
"""W 21 St & 6 Av…",247908
"""West St & Cham…",219580
"""1 Ave & E 68 S…",208483
"""6 Ave & W 33 S…",197135
"""Broadway & W 2…",194687


There are some junk stations in the dataset:

In [42]:
q=(
    group1_pl.lazy()
    .groupby('start_station_name')
    .agg(pl.count('start_station_name').alias('count'))
    .filter(
        (pl.col('count')>=10)
    )
    .sort('count',descending=True)
    # .limit(5)
)

group1_pl_count=q.collect()
print(group1_pl_count)

shape: (2_027, 2)
┌──────────────────────────┬────────┐
│ start_station_name       ┆ count  │
│ ---                      ┆ ---    │
│ str                      ┆ u32    │
╞══════════════════════════╪════════╡
│ W 21 St & 6 Ave          ┆ 247908 │
│ West St & Chambers St    ┆ 219580 │
│ 1 Ave & E 68 St          ┆ 208483 │
│ 6 Ave & W 33 St          ┆ 197135 │
│ …                        ┆ …      │
│ Prototype Lab            ┆ 15     │
│ Apache                   ┆ 15     │
│ 4455.10                  ┆ 11     │
│ Rogers Pl & E 165 St_old ┆ 11     │
└──────────────────────────┴────────┘


However, these junk stations are such a small part of the dataset that I think it's ok to include.  
When it comes time to query the database with `SQL`, we can work to remove these junk stations with more control.

In [43]:
q=(
    group1_pl.lazy()
    .groupby('start_station_id')
    .agg(pl.count('start_station_id').alias('count'))
    # .filter(
        # (pl.col('count')>=1)
    # )
    .sort('count',descending=True)
    # .limit(5)
)

group1_pl_count=q.collect()
print(group1_pl_count)

shape: (1_927, 2)
┌──────────────────┬────────┐
│ start_station_id ┆ count  │
│ ---              ┆ ---    │
│ f64              ┆ u32    │
╞══════════════════╪════════╡
│ 6140.05          ┆ 247908 │
│ 5329.03          ┆ 219580 │
│ 6822.09          ┆ 208483 │
│ 6364.07          ┆ 197135 │
│ …                ┆ …      │
│ 5548.01          ┆ 44     │
│ 3704.01          ┆ 33     │
│ 4014.01          ┆ 30     │
│ 8419.03          ┆ 2      │
└──────────────────┴────────┘


Practice making new columns of summary information using other columns:

In [44]:
q=(
    group1_pl.lazy()
    .groupby('rideable_type')
    .agg(pl.count('rideable_type').alias('rideable_type_count'))
    .sort('rideable_type_count',descending=True)
)

group1_pl_rideable_type=q.collect()

q=(
    group1_pl_rideable_type.lazy()
    .with_columns([pl.col("rideable_type_count").sum().alias('sum')])
)

group1_pl_rideable_type=q.collect()

print(group1_pl_rideable_type)

shape: (3, 3)
┌───────────────┬─────────────────────┬──────────┐
│ rideable_type ┆ rideable_type_count ┆ sum      │
│ ---           ┆ ---                 ┆ ---      │
│ str           ┆ u32                 ┆ u32      │
╞═══════════════╪═════════════════════╪══════════╡
│ classic_bike  ┆ 34010263            ┆ 55800085 │
│ electric_bike ┆ 13092408            ┆ 55800085 │
│ docked_bike   ┆ 8697414             ┆ 55800085 │
└───────────────┴─────────────────────┴──────────┘


There are about three times as many classic bike trips used than electric bikes used. The total amount of trips equals the total rows of the dataset.

In [45]:
q=(
    group1_pl.lazy()
    .groupby('member_casual')
    .agg(pl.count('member_casual').alias('member_casual_count'))
    .sort('member_casual_count',descending=True)
)

group1_pl_rider_type=q.collect()

print(group1_pl_rider_type)

shape: (2, 2)
┌───────────────┬─────────────────────┐
│ member_casual ┆ member_casual_count │
│ ---           ┆ ---                 │
│ str           ┆ u32                 │
╞═══════════════╪═════════════════════╡
│ member        ┆ 42560681            │
│ casual        ┆ 13239404            │
└───────────────┴─────────────────────┘


#### Determine the number and contents of secondary tables that are needed.

A `member_casual` table that has two rows, `member` and `casual`:

In [46]:
member_casual=group1_pl['member_casual'].value_counts().sort('counts',descending=True)
member_casual

member_casual,counts
str,u32
"""member""",42560681
"""casual""",13239404


In [47]:
sum(member_casual['counts'])

55800085

A `rideable_type` table that has three rows, `classic_bike`, `electric_bike`, and `docked_bike`:

In [48]:
group1_pl['rideable_type'].value_counts().sort('counts',descending=True)

rideable_type,counts
str,u32
"""classic_bike""",34010263
"""electric_bike""",13092408
"""docked_bike""",8697414


There are 1,926 stations in the database, plus a `null` entry:

In [49]:
group1_pl['start_station_id'].value_counts().sort('start_station_id',descending=True)

start_station_id,counts
f64,u32
8841.03,4503
8811.01,2113
8799.01,3221
8795.03,2011
8795.01,6404
8782.01,3084
8778.01,2968
8766.03,2209
8752.01,2996


## Normalize `group1`

### Ride table:

#### Normalize the `rideable_type` and `member_casual` categories:

In [50]:
group1_pl_intermediate=\
group1_pl.with_columns(
    pl.when(pl.col('member_casual') == 'member')
    .then(pl.lit(0))
    .otherwise(pl.lit(1))
    .alias('ridertype'))

group1_pl_intermediate.head()

# named intermediate because this is an intermediate step
# before finalizing this table of data

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ridertype
str,str,datetime[μs],datetime[μs],str,f64,str,f64,f64,f64,f64,f64,str,i32
"""26A3DC47FE0EA3…","""docked_bike""",2021-05-13 12:48:08,2021-05-13 13:07:37,"""Broadway & W 2…",6173.08,"""E 2 St & Avenu…",5515.02,40.742868,-73.989186,40.722174,-73.983688,"""member""",0
"""A99F2E1D627B08…","""docked_bike""",2021-05-16 08:30:13,2021-05-16 08:45:47,"""46 Ave & 5 St""",6286.02,"""34th Ave & Ver…",6873.01,40.74731,-73.95451,40.765354,-73.939863,"""member""",0
"""43E79A45997B73…","""docked_bike""",2021-05-01 08:38:14,2021-05-01 08:54:27,"""46 Ave & 5 St""",6286.02,"""34th Ave & Ver…",6873.01,40.74731,-73.95451,40.765354,-73.939863,"""member""",0
"""8B3CC649F4F588…","""docked_bike""",2021-05-09 08:12:31,2021-05-09 08:27:05,"""46 Ave & 5 St""",6286.02,"""34th Ave & Ver…",6873.01,40.74731,-73.95451,40.765354,-73.939863,"""member""",0
"""7101C98F057486…","""docked_bike""",2021-05-27 07:52:27,2021-05-27 08:09:01,"""E 123 St & Lex…",7636.05,"""1 Ave & E 78 S…",7020.09,40.802926,-73.9379,40.771404,-73.953517,"""member""",0


In [51]:
# sanity check on the column that was just created
group1_pl_intermediate['ridertype'].value_counts().sort('counts',descending=True)

ridertype,counts
i32,u32
0,42560681
1,13239404


In [52]:
group1_pl_intermediate=\
group1_pl_intermediate.with_columns(
        pl.when(pl.col('rideable_type') == 'classic_bike')
        .then(pl.lit(0))
        .when(pl.col('rideable_type') == 'electric_bike')
        .then(pl.lit(1))
        .otherwise(pl.lit(2))
        .alias('biketype')
)

group1_pl_intermediate.head()

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ridertype,biketype
str,str,datetime[μs],datetime[μs],str,f64,str,f64,f64,f64,f64,f64,str,i32,i32
"""26A3DC47FE0EA3…","""docked_bike""",2021-05-13 12:48:08,2021-05-13 13:07:37,"""Broadway & W 2…",6173.08,"""E 2 St & Avenu…",5515.02,40.742868,-73.989186,40.722174,-73.983688,"""member""",0,2
"""A99F2E1D627B08…","""docked_bike""",2021-05-16 08:30:13,2021-05-16 08:45:47,"""46 Ave & 5 St""",6286.02,"""34th Ave & Ver…",6873.01,40.74731,-73.95451,40.765354,-73.939863,"""member""",0,2
"""43E79A45997B73…","""docked_bike""",2021-05-01 08:38:14,2021-05-01 08:54:27,"""46 Ave & 5 St""",6286.02,"""34th Ave & Ver…",6873.01,40.74731,-73.95451,40.765354,-73.939863,"""member""",0,2
"""8B3CC649F4F588…","""docked_bike""",2021-05-09 08:12:31,2021-05-09 08:27:05,"""46 Ave & 5 St""",6286.02,"""34th Ave & Ver…",6873.01,40.74731,-73.95451,40.765354,-73.939863,"""member""",0,2
"""7101C98F057486…","""docked_bike""",2021-05-27 07:52:27,2021-05-27 08:09:01,"""E 123 St & Lex…",7636.05,"""1 Ave & E 78 S…",7020.09,40.802926,-73.9379,40.771404,-73.953517,"""member""",0,2


In [53]:
# sanity check on the column that was just created
group1_pl_intermediate['biketype'].value_counts().sort('counts',descending=True)

biketype,counts
i32,u32
0,34010263
1,13092408
2,8697414


So far so good. Now to finalize the `ride` table.

In [54]:
group1_pl_ridenorm=group1_pl_intermediate.select(pl.col('*').exclude(\
'rideable_type',
'member_casual',
'start_station_name',
'end_station_name',
'start_lat',
'end_lat',
'start_lng',
'end_lng'))

group1_pl_ridenorm.head()

ride_id,started_at,ended_at,start_station_id,end_station_id,ridertype,biketype
str,datetime[μs],datetime[μs],f64,f64,i32,i32
"""26A3DC47FE0EA3…",2021-05-13 12:48:08,2021-05-13 13:07:37,6173.08,5515.02,0,2
"""A99F2E1D627B08…",2021-05-16 08:30:13,2021-05-16 08:45:47,6286.02,6873.01,0,2
"""43E79A45997B73…",2021-05-01 08:38:14,2021-05-01 08:54:27,6286.02,6873.01,0,2
"""8B3CC649F4F588…",2021-05-09 08:12:31,2021-05-09 08:27:05,6286.02,6873.01,0,2
"""7101C98F057486…",2021-05-27 07:52:27,2021-05-27 08:09:01,7636.05,7020.09,0,2


In [55]:
# sanity check on the table that was just created
group1_pl_ridenorm['biketype'].value_counts().sort('counts',descending=True)

biketype,counts
i32,u32
0,34010263
1,13092408
2,8697414


### Station table:

In [56]:
group1_pl_stationnorm=group1_pl.\
select(pl.col('*')\
       .exclude('ride_id',
                'rideable_type',
                'started_at',
                'ended_at',
                'member_casual',
                'end_station_name',
                'end_station_id',
                'end_lat',
                'end_lng'
               ))

group1_pl_stationnorm.head()

start_station_name,start_station_id,start_lat,start_lng
str,f64,f64,f64
"""Broadway & W 2…",6173.08,40.742868,-73.989186
"""46 Ave & 5 St""",6286.02,40.74731,-73.95451
"""46 Ave & 5 St""",6286.02,40.74731,-73.95451
"""46 Ave & 5 St""",6286.02,40.74731,-73.95451
"""E 123 St & Lex…",7636.05,40.802926,-73.9379


I want to check to make sure that all the `station_id` are accounted for in the `start_station_id` column:

In [58]:
# check if all values in column_a are represented in column_b
all_values_included = (group1_pl['start_station_id'].is_in(group1_pl['end_station_id'])).all()

# print the result
print(all_values_included)

True


## Convert the `polars` tables into a form that will make it easy to convert to `MySQL`.

This is because the size of the tables warrants compression.

Save the following tables. 
* `group1_pl_ridenorm`
* `group1_pl_stationnorm`

They are currently in a `polars` data type:

In [14]:
type(group1_pl_ridenorm)

polars.dataframe.frame.DataFrame

We will convert the `polars` dataframe to a `parquet` dataframe before sending to `MySQL`:

In [59]:
path_to_group1_pl_ridenorm_parquet=\
'/Users/sra/files/projects/citibike_project/combined/group1_combined/group1_pl_ridenorm.parquet'

if not os.path.exists(path_to_group1_pl_ridenorm_parquet):
    group1_pl_ridenorm.write_parquet(path_to_group1_pl_ridenorm_parquet)

In [60]:
path_to_group1_pl_stationnorm_parquet=\
'/Users/sra/files/projects/citibike_project/combined/group1_combined/group1_pl_stationnorm.parquet'

if not os.path.exists(path_to_group1_pl_stationnorm_parquet):
    group1_pl_stationnorm.write_parquet(path_to_group1_pl_stationnorm_parquet)

Now we can proceed to create the `SQL` tables for **group1**!

# `group2`

In [71]:
from pathlib import Path

import pandas as pd
import numpy as np

import polars as pl

In [3]:
group2_full_location=Path('/Users/sra/files/projects/citibike_project/combined/group2_pl_full.parquet')

group2=pl.read_parquet(group2_full_location)

In [14]:
group2.shape

(225093064, 15)

In [4]:
group2.sample(n=10)

trip_duration,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,bike_id,user_type,birth_year,gender
f64,datetime[μs],datetime[μs],f64,str,f64,f64,f64,str,f64,f64,f64,str,f64,f64
565.0,null,null,363.0,"""West Thames St…",40.708347,-74.017134,368.0,"""Carmine St & 6…",40.730386,-74.00215,21521.0,"""Subscriber""",1989.0,1.0
375.0,2017-07-02 01:24:25,2017-07-02 01:30:41,369.0,"""Washington Pl …",40.732241,-74.000264,3263.0,"""Cooper Square …",40.729236,-73.990868,28357.0,"""Subscriber""",1977.0,1.0
2199.0,null,null,368.0,"""Carmine St & 6…",40.730386,-74.00215,369.0,"""Washington Pl …",40.732241,-74.000264,43582.0,"""Subscriber""",1986.0,1.0
869.0,null,null,164.0,"""E 47 St & 2 Av…",40.753231,-73.970325,251.0,"""Mott St & Prin…",40.72318,-73.9948,26899.0,"""Subscriber""",1984.0,1.0
2834.0,null,null,281.0,"""Grand Army Pla…",40.764397,-73.973715,2006.0,"""Central Park S…",40.765909,-73.976342,17351.0,"""Customer""",null,0.0
697.0,null,null,357.0,"""E 11 St & Broa…",40.732618,-73.99158,402.0,"""Broadway & E 2…",40.740343,-73.989551,32683.0,"""Subscriber""",1972.0,2.0
319.0,null,null,259.0,"""South St & Whi…",40.701221,-74.012342,279.0,"""Peck Slip & Fr…",40.707873,-74.00167,18590.0,"""Subscriber""",1977.0,1.0
516.0,null,null,195.0,"""Liberty St & B…",40.709056,-74.010434,377.0,"""6 Ave & Canal …",40.722438,-74.005664,31783.0,"""Subscriber""",1975.0,1.0
619.0,2017-07-06 15:21:52,2017-07-06 15:32:11,479.0,"""9 Ave & W 45 S…",40.760193,-73.991255,3255.0,"""8 Ave & W 31 S…",40.750585,-73.994685,14816.0,"""Customer""",null,0.0


In [5]:
group2.groupby('start_station_name').count()\
.sort('count',descending=True).head(10)

start_station_name,count
str,u32
null,11663330
"""Pershing Squar…",1543232
"""E 17 St & Broa…",1438584
"""W 21 St & 6 Av…",1383136
"""West St & Cham…",1327624
"""8 Ave & W 31 S…",1274724
"""Broadway & E 2…",1271352
"""Broadway & E 1…",1207014
"""Lafayette St &…",1176504


`filter` to remove any `None` values in the 'started_at' or 'ended_at' columns

In [6]:
group2_no_datetime_nulls=group2.filter(pl.col('started_at') != None)
group2_no_datetime_nulls=group2_no_datetime_nulls.filter(pl.col('ended_at') != None)
group2_no_datetime_nulls.sample(10)

trip_duration,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,bike_id,user_type,birth_year,gender
f64,datetime[μs],datetime[μs],f64,str,f64,f64,f64,str,f64,f64,f64,str,f64,f64
668.0,2017-11-17 13:56:40,2017-11-17 14:07:49,527.0,"""E 33 St & 2 Av…",40.744023,-73.976056,477.0,"""W 41 St & 8 Av…",40.756405,-73.990026,17858.0,"""Subscriber""",1989.0,1.0
695.0,2017-09-29 16:16:08,2017-09-29 16:27:43,270.0,"""Adelphi St & M…",40.693083,-73.971789,3415.0,"""Prospect Pl & …",40.679331,-73.975195,31468.0,"""Subscriber""",1987.0,2.0
1600.0,2017-06-13 22:36:22,2017-06-13 23:03:02,3142.0,"""1 Ave & E 62 S…",40.761227,-73.96094,446.0,"""W 24 St & 7 Av…",40.744876,-73.995299,28520.0,"""Subscriber""",1970.0,2.0
579.0,2017-12-07 18:46:29,2017-12-07 18:56:09,528.0,"""2 Ave & E 31 S…",40.742909,-73.977061,435.0,"""W 21 St & 6 Av…",40.74174,-73.994156,32026.0,"""Subscriber""",1991.0,1.0
371.0,2017-09-05 09:08:28,2017-09-05 09:14:39,498.0,"""Broadway & W 3…",40.748549,-73.988084,536.0,"""1 Ave & E 30 S…",40.741444,-73.975361,15231.0,"""Subscriber""",1988.0,1.0
265.0,2017-10-30 09:01:07,2017-10-30 09:05:32,3580.0,"""St Johns Pl & …",40.673724,-73.963161,3576.0,"""Park Pl & Vand…",40.6767,-73.969024,21422.0,"""Subscriber""",1986.0,2.0
365.0,2017-06-12 08:05:13,2017-06-12 08:11:18,442.0,"""W 27 St & 7 Av…",40.746647,-73.993915,402.0,"""Broadway & E 2…",40.740343,-73.989551,27758.0,"""Subscriber""",1961.0,1.0
284.0,2017-06-08 08:45:02,2017-06-08 08:49:47,3088.0,"""Union Ave & Ja…",40.716075,-73.952029,3016.0,"""Kent Ave & N 7…",40.720368,-73.961651,28349.0,"""Subscriber""",1984.0,1.0
212.0,2017-07-15 12:35:25,2017-07-15 12:38:58,3159.0,"""W 67 St & Broa…",40.774925,-73.982666,3173.0,"""Riverside Blvd…",40.777507,-73.988886,18722.0,"""Subscriber""",1987.0,2.0


In [8]:
group2_no_datetime_nulls.shape

(39306552, 15)

In [12]:
group2_no_datetime_nulls.null_count()

trip_duration,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,bike_id,user_type,birth_year,gender
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,4675892,0


In [48]:
print(f'The percentage of nulls in birth year \n\
to total count of rows in the dataset is: \
{round((group2_no_datetime_nulls["birth_year"].null_count() / group2_no_datetime_nulls.shape[0])* 100, 2)}%')

The percentage of nulls in birth year 
to total count of rows in the dataset is: 11.9%


In [30]:
group2_no_datetime_nulls.groupby('birth_year').count()\
.filter(pl.col('birth_year') >= 1923)\
.sort('birth_year',descending=True).head(10)

birth_year,count
f64,u32
2001.0,10310
2000.0,31750
1999.0,61742
1998.0,107064
1997.0,142054
1996.0,198332
1995.0,274084
1994.0,436230
1993.0,672594


`filter` to remove any `None` values in the 'birth_year' column:

In [51]:
group2_no_nulls=group2_no_datetime_nulls.filter(pl.col('birth_year') != None)
group2_no_nulls.shape

(34630660, 15)

In [70]:
print(f"The ratio of non-subscribers to subscribers is: \n\
{round(sum(group2_no_nulls['user_type']!='Subscriber') / sum(group2_no_nulls['user_type']=='Subscriber'),2) * 100}%")

The ratio of non-subscribers to subscribers is: 
1.0%


What I want to know is if there is anything explaining the vast amount of 'nulls' in the 'started_at' and 'ended_at' columns for `group2`

First, save the no_nulls dataframe to parquet as a way to save our progress:

In [74]:
group2_no_nulls_location=Path('/Users/sra/files/projects/citibike_project/combined/group2_no_nulls.parquet')

group2_no_nulls.write_parquet(group2_no_nulls_location)

In [7]:
import seaborn as sns
import pandas as pd
import polars as pl
import numpy as np

group2_no_nulls_location='/Users/sra/files/projects/citibike_project/combined/group2_no_nulls.parquet'

Open the no_nulls parquet file and save it as a pandas dataframe so that seaborn can manipulate it:

In [8]:
group2_no_nulls_pd = pd.read_parquet(path=group2_no_nulls_location)

In [81]:
group2_no_nulls_pd.sample(5)

,trip_duration,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,bike_id,user_type,birth_year,gender
5139712,592.0,2014-08-28 20:51:16,2014-08-28 21:01:08,323.0,Lawrence St & Willoughby St,40.692362,-73.986317,395.0,Bond St & Schermerhorn St,40.688070,-73.984106,21180.0,Subscriber,1964.0,1.0
4079619,589.0,2015-03-08 18:27:37,2015-03-08 18:37:26,388.0,W 26 St & 10 Ave,40.749718,-74.002950,463.0,9 Ave & W 16 St,40.742065,-74.004432,17181.0,Subscriber,1972.0,2.0
24482056,340.0,2017-07-17 08:48:39,2017-07-17 08:54:20,3151.0,E 81 St & York Ave,40.772838,-73.949892,3156.0,E 72 St & York Ave,40.766638,-73.953483,19943.0,Subscriber,1981.0,1.0
11159419,792.0,2017-12-13 07:46:03,2017-12-13 07:59:16,3223.0,E 55 St & 3 Ave,40.758997,-73.968654,477.0,W 41 St & 8 Ave,40.756405,-73.990026,26406.0,Subscriber,1997.0,1.0
23116374,524.0,2017-11-14 09:21:09,2017-11-14 09:29:53,482.0,W 15 St & 7 Ave,40.739355,-73.999318,472.0,E 32 St & Park Ave,40.745712,-73.981948,25338.0,Subscriber,1983.0,1.0


Now make a `sns.pairplot` to explore:

In [ ]:
sns.pairplot(group2_no_nulls_pd)